In [1]:
!pip install mwclient

In [1]:
import mwclient
import time

site = mwclient.Site("en.wikipedia.org")
page = site.pages["Bitcoin"]

In [2]:
revs = list(page.revisions())

In [3]:
revs[0]

OrderedDict([('revid', 1331337329),
             ('parentid', 1331274620),
             ('user', 'Jtbobwaysf'),
             ('timestamp',
              time.struct_time(tm_year=2026, tm_mon=1, tm_mday=5, tm_hour=21, tm_min=24, tm_sec=50, tm_wday=0, tm_yday=5, tm_isdst=-1)),
             ('comment',
              'Undid revision [[Special:Diff/1331274620|1331274620]] by [[Special:Contributions/FlippyFlink|FlippyFlink]] ([[User talk:FlippyFlink|talk]]) not an improvement')])

In [4]:
revs = sorted(revs, key=lambda rev: rev["timestamp"])

In [5]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [7]:
!pip install transformers
!pip install torch

In [6]:
import time
import torch
from transformers import pipeline
from collections import defaultdict
from statistics import mean

In [19]:
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline(text[:250])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [20]:
total = len(revs)
valid = sum(
    1 for rev in revs
    if isinstance(rev.get("comment"), str) and rev["comment"].strip()
)

print(f"Total revs: {total}")
print(f"Valid comments: {valid}")
print(f"Removed: {total - valid}")


Total revs: 18109
Valid comments: 16866
Removed: 1243


In [21]:
clean_revs = [
    rev for rev in revs
    if isinstance(rev.get("comment"), str) and rev["comment"].strip()
]

In [22]:
edits = {} 

for rev in clean_revs: 
    date = time.strftime("%Y-%m-%d", rev["timestamp"]) 
    
    if date not in edits: 
        edits[date] = dict(sentiments=list(), edit_count = 0) 
        
    edits[date]["edit_count"] += 1 

    comment = rev["comment"]
    edits[date]["sentiments"].append(find_sentiment(comment))

In [23]:
from statistics import mean 
for key in edits: 
    if len(edits[key]["sentiments"]) > 0: 
        edits[key]["sentiment"] = mean(edits[key]["sentiments"]) 
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s<0]) / len(edits[key]["sentiments"]) 
    else: 
        edits[key]["sentiment"] = 0 
        edits[key]["neg_sentiment"] = 0 
    del edits[key]["sentiments"]

In [24]:
edits

{'2009-03-08': {'edit_count': 3,
  'sentiment': -0.9834070007006327,
  'neg_sentiment': 1.0},
 '2009-08-06': {'edit_count': 2,
  'sentiment': 0.9957457184791565,
  'neg_sentiment': 0.0},
 '2009-08-14': {'edit_count': 1,
  'sentiment': 0.9300208687782288,
  'neg_sentiment': 0.0},
 '2009-10-13': {'edit_count': 2,
  'sentiment': -0.2275010049343109,
  'neg_sentiment': 0.5},
 '2009-11-18': {'edit_count': 1,
  'sentiment': 0.8839505910873413,
  'neg_sentiment': 0.0},
 '2009-12-08': {'edit_count': 1,
  'sentiment': -0.9869275689125061,
  'neg_sentiment': 1.0},
 '2009-12-17': {'edit_count': 1,
  'sentiment': -0.9975171089172363,
  'neg_sentiment': 1.0},
 '2010-02-23': {'edit_count': 1,
  'sentiment': -0.9994946718215942,
  'neg_sentiment': 1.0},
 '2010-03-18': {'edit_count': 1,
  'sentiment': 0.8758775591850281,
  'neg_sentiment': 0.0},
 '2010-04-13': {'edit_count': 4,
  'sentiment': 0.8443557620048523,
  'neg_sentiment': 0.0},
 '2010-04-15': {'edit_count': 5,
  'sentiment': 0.476089739799499

In [25]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [26]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,3,-0.983407,1.0
2009-08-06,2,0.995746,0.0
2009-08-14,1,0.930021,0.0
2009-10-13,2,-0.227501,0.5
2009-11-18,1,0.883951,0.0
...,...,...,...
2025-12-17,2,-0.992427,1.0
2025-12-18,2,-0.999133,1.0
2025-12-25,1,-0.786758,1.0
2026-01-04,1,-0.981123,1.0


In [27]:
edits_df.index = pd.to_datetime(edits_df.index)

In [28]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08",end="2026-01-05")

In [29]:
dates

DatetimeIndex(['2009-03-08', '2009-03-09', '2009-03-10', '2009-03-11',
               '2009-03-12', '2009-03-13', '2009-03-14', '2009-03-15',
               '2009-03-16', '2009-03-17',
               ...
               '2025-12-27', '2025-12-28', '2025-12-29', '2025-12-30',
               '2025-12-31', '2026-01-01', '2026-01-02', '2026-01-03',
               '2026-01-04', '2026-01-05'],
              dtype='datetime64[ns]', length=6148, freq='D')

In [30]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [31]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,3,-0.983407,1.0
2009-03-09,0,0.000000,0.0
2009-03-10,0,0.000000,0.0
2009-03-11,0,0.000000,0.0
2009-03-12,0,0.000000,0.0
...,...,...,...
2026-01-01,0,0.000000,0.0
2026-01-02,0,0.000000,0.0
2026-01-03,0,0.000000,0.0
2026-01-04,1,-0.981123,1.0


In [32]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()

In [33]:
rolling_edits = rolling_edits.dropna()

In [34]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.100000,-0.032780,0.033333
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2026-01-01,0.200000,-0.125924,0.133333
2026-01-02,0.200000,-0.125924,0.133333
2026-01-03,0.200000,-0.125924,0.133333
2026-01-04,0.233333,-0.158628,0.166667


In [35]:
rolling_edits.to_csv("wikipedia_edits.csv")